# Sentiment Analysis Notebook

## Dataset

The data given is in the form of a comma-separated values files with tweets and their corresponding sentiments. The training dataset is a csv file of type tweet_id,sentiment,tweet where the
tweet_id is a unique integer identifying the tweet, sentiment is either 1 (positive) or 0 (negative), and tweet is the tweet enclosed in "". Similarly, the test dataset is a csv file of type
tweet_id,tweet.

In [1]:
import pandas as pd
import sys
import nltk
sys.path.insert(0, 'src')

In [ ]:
nltk.download()

In [2]:
dataset = pd.read_csv('dataset/train.csv', encoding = "ISO-8859-1")
dataset.head(5)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


## Preprocessing

### Preprocessing functions

In [3]:
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def preprocess_word(word):
    # remove punctation
    word = word.strip('\'"?!,.():;*')
    # more than 3 letter repetition removed
    word = re.sub(r'(.)\1\1+', r'\1\1\1', word)
    # remove - & '
    word = word.strip('-&\'')
    return word

def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def preprocess_tweet(tweet, use_stemmer=False, use_lemmatizer=False):
    # convert tweet to lowercase
    tweet = tweet.lower()
    # replace urls with 'URL'
    tweet = re.sub(r'((www.[\S]+)|(https?://.[\S]+))', 'URL', tweet)
    # replace user mentions @user with 'USER_MENTION'
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # replace #hashtag with hastag
    tweet = re.sub(r'#(\S+)', r' \1', tweet)
    # remove retweet RT
    tweet = re.sub(r'\brt\b', '', tweet)
    # replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # remove space, " and ' 
    tweet.strip('" \'')
    # handle emojis. Use only EMO_POS and EMO_NEG
    tweet = handle_emojis(tweet)
    # replace multiple spaces with only one space
    tweet = re.sub(r'\s+', ' ', tweet)
    # preprocess words
    words = tweet.split()

    processed_words = []
    porter_stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                # use stemmer
                word = str(porter_stemmer.stem(word))
            elif use_lemmatizer:
                word = str(wordnet_lemmatizer.lemmatize(word))
            processed_words.append(word)
    return ' '.join(processed_words)

In [4]:
#tweet = "Hi, how is going on? :) CIAO COME vaaaaa? :) :( https://tartarus.org/martin/PorterStemmer/  bella @ivocerti    #sentimentanalysis"
for i in range(4):
    tweet = dataset.loc[i]['SentimentText']
    print('ORIGINAL: ' + tweet)
    print('STEMMER: ' + preprocess_tweet(tweet, use_stemmer=True))
    print('LEMMATIZER: ' + preprocess_tweet(tweet, use_lemmatizer=True))
    print('----------------------------------------------------------------')

ORIGINAL:                      is so sad for my APL friend.............
STEMMER: is so sad for my apl friend
LEMMATIZER: is so sad for my apl friend
----------------------------------------------------------------
ORIGINAL:                    I missed the New Moon trailer...
STEMMER: i miss the new moon trailer
LEMMATIZER: i missed the new moon trailer
----------------------------------------------------------------
ORIGINAL:               omg its already 7:30 :O
STEMMER: omg it alreadi o
LEMMATIZER: omg it already o
----------------------------------------------------------------
ORIGINAL:           .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...
STEMMER: omgaga im sooo im gunna cri been at thi dentist sinc i wa supos just get a crown put on
LEMMATIZER: omgaga im sooo im gunna cry been at this dentist since i wa suposed just get a crown put on
----------------------------------------------------------------


### Preprocess the whole dataset

In [ ]:
preprocessed_train_file = open('dataset/preprocessed_train.csv', 'w')
preprocessed_train_file.write('ItemID,Sentiment,SentimentText\n')
for index, row in dataset.iterrows():
    res = preprocess_tweet(row['SentimentText'])
    preprocessed_train_file.write(str(index+1) + ',' + str(row['Sentiment']) + ',' + preprocess_tweet(row['SentimentText']) + '\n')
    sys.stdout.write('\r')
    sys.stdout.write('Processing ' + str(index+1) + '/' + str(dataset.shape[0]))
    sys.stdout.flush()
preprocessed_train_file.close()

In [5]:
preprocessed_dataset = pd.read_csv('dataset/preprocessed_train.csv', encoding = "ISO-8859-1")
preprocessed_dataset.head(5)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl friend
1,2,0,i missed the new moon trailer
2,3,1,omg its already o
3,4,0,omgaga im sooo im gunna cry been at this denti...
4,5,0,i think mi bf is cheating on me t_t


### Feature parameters (Word, SentiWordNet, POS + POS_Word, Semantic)

In [6]:
from nltk.corpus import sentiwordnet as swn

def word_to_SWNt(word):
    # take the first result from sentiwordnet (if there is)
    list_sent = list(swn.senti_synsets(word))
    if len(list_sent) > 0:
        sent = list_sent[0]
        pos = round(sent.pos_score()*10)
        neg = round(sent.neg_score()*10)
        obj = round(sent.obj_score()*10)
        # return the word if the word itself is mostly object
        # otherwise return POS-X or NEG-X if the word is 
        # mostly positive or negative and X is the rounded score*10 (from 0 to 10)
        if pos > neg:
            return 'POS-' + str(pos)
        elif neg > pos:
            return 'NEG-' + str(neg)
    return word

def tweet_to_feature_word(tweet):
    # just return the tweet as it is
    return str(tweet)

def tweet_to_feature_SWNt(tweet):
    words = str(tweet).split()
    processed_words = []
    for word in words:
        processed_words.append(word_to_SWNt(word))
    return str(' '.join(processed_words))

def tweet_to_feature_POS(tweet):
    # the tweet is composed by the POS (parts of speech) of the words
    # first just the list of pos, then the list of the pos and the words
    # in the format pos_word
    words = str(tweet).split()
    words_POS = nltk.pos_tag(words)
    processed_words = []
    pos = []
    for word in words_POS:
        pos.append(word[1])
    processed_words = pos
    size = len(processed_words)
    for i in range(size):
        processed_words.append(processed_words[i] + '_' + words[i])
    return str(' '.join(processed_words))

def tweet_to_feature_semantic(tweet):
    return tweet

In [7]:
for i in range(4):
    tweet = preprocessed_dataset.loc[i]['SentimentText']
    print('WORD: ' + tweet_to_feature_word(tweet))
    print('SWNt: ' + tweet_to_feature_SWNt(tweet))
    print('POS+POS_word: ' + tweet_to_feature_POS(tweet))
    #print('Semantic: ' + tweet_to_feature_semantic(tweet))
    print('----------------------------------------------------------------')

WORD: is so sad for my apl friend
SWNt: POS-2 so NEG-8 for my apl POS-1
POS+POS_word: VBZ RB JJ IN PRP$ NN NN VBZ_is RB_so JJ_sad IN_for PRP$_my NN_apl NN_friend
----------------------------------------------------------------
WORD: i missed the new moon trailer
SWNt: i NEG-2 the POS-4 moon trailer
POS+POS_word: NN VBD DT JJ NN NN NN_i VBD_missed DT_the JJ_new NN_moon NN_trailer
----------------------------------------------------------------
WORD: omg its already o
SWNt: omg its POS-1 o
POS+POS_word: VB PRP$ RB VB VB_omg PRP$_its RB_already VB_o
----------------------------------------------------------------
WORD: omgaga im sooo im gunna cry been at this dentist since i was suposed just get a crown put on
SWNt: omgaga im sooo im gunna cry POS-2 at this dentist since i was suposed POS-6 get a crown put on
POS+POS_word: JJ NN NN NN NN NN VBN IN DT NN IN NN VBD VBN RB VB DT NN NN IN JJ_omgaga NN_im NN_sooo NN_im NN_gunna NN_cry VBN_been IN_at DT_this NN_dentist IN_since NN_i VBD_was VBN

### Save the whole datasets

In [189]:
# WORD
features_train_file = open('dataset/word_features_train.csv', 'w')
features_train_file.write('ItemID,Sentiment,SentimentText\n')
print('WORD')
for index, row in preprocessed_dataset.iterrows():
    features_train_file.write(str(index+1) + ',' + str(row['Sentiment']) + ',' + tweet_to_feature_word(row['SentimentText']) + '\n')
    sys.stdout.write('\r')
    sys.stdout.write('Processing ' + str(index+1) + '/' + str(preprocessed_dataset.shape[0]))
    sys.stdout.flush()
preprocessed_train_file.close()

# SWNt
features_train_file = open('dataset/swnt_features_train.csv', 'w')
features_train_file.write('ItemID,Sentiment,SentimentText\n')
print('\nSWNt')
for index, row in preprocessed_dataset.iterrows():
    features_train_file.write(str(index+1) + ',' + str(row['Sentiment']) + ',' + tweet_to_feature_SWNt(row['SentimentText']) + '\n')
    sys.stdout.write('\r')
    sys.stdout.write('Processing ' + str(index+1) + '/' + str(preprocessed_dataset.shape[0]))
    sys.stdout.flush()
preprocessed_train_file.close()

# POS+POS_word
features_train_file = open('dataset/pos_features_train.csv', 'w')
features_train_file.write('ItemID,Sentiment,SentimentText\n')
print('\nSWNt')
for index, row in preprocessed_dataset.iterrows():
    features_train_file.write(str(index+1) + ',' + str(row['Sentiment']) + ',' + tweet_to_feature_POS(row['SentimentText']) + '\n')
    sys.stdout.write('\r')
    sys.stdout.write('Processing ' + str(index+1) + '/' + str(preprocessed_dataset.shape[0]))
    sys.stdout.flush()
preprocessed_train_file.close()

WORD
Processing 99989/99989Processing 11364/99989Processing 12648/99989Processing 32953/99989Processing 63347/99989Processing 64747/99989Processing 67747/99989Processing 73962/99989
SWNt
Processing 99989/99989
SWNt
Processing 99989/99989

In [8]:
# WORD
features_word_dataset = pd.read_csv('dataset/word_features_train.csv', encoding = "ISO-8859-1")
print('WORD')
features_word_dataset.head(5)

WORD


,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl friend
1,2,0,i missed the new moon trailer
2,3,1,omg its already o
3,4,0,omgaga im sooo im gunna cry been at this denti...
4,5,0,i think mi bf is cheating on me t_t


In [9]:
# SWNt
features_swnt_dataset = pd.read_csv('dataset/swnt_features_train.csv', encoding = "ISO-8859-1")
print('SWNt')
features_swnt_dataset.head(5)

SWNt


,ItemID,Sentiment,SentimentText
0,1,0,POS-2 so NEG-8 for my apl POS-1
1,2,0,i NEG-2 the POS-4 moon trailer
2,3,1,omg its POS-1 o
3,4,0,omgaga im sooo im gunna cry POS-2 at this dent...
4,5,0,i think mi bf POS-2 POS-6 on me t_t


In [10]:
# POS+POS_word
features_pos_dataset = pd.read_csv('dataset/pos_features_train.csv', encoding = "ISO-8859-1")
print('POS+POS_word')
features_pos_dataset.head(5)

POS+POS_word


,ItemID,Sentiment,SentimentText
0,1,0,VBZ RB JJ IN PRP$ NN NN VBZ_is RB_so JJ_sad IN...
1,2,0,NN VBD DT JJ NN NN NN_i VBD_missed DT_the JJ_n...
2,3,1,VB PRP$ RB VB VB_omg PRP$_its RB_already VB_o
3,4,0,JJ NN NN NN NN NN VBN IN DT NN IN NN VBD VBN R...
4,5,0,NN VBP NN NN VBZ VBG IN PRP VB NN_i VBP_think ...


## Vectorization

### Dictionary

In [11]:
from nltk import FreqDist
import pickle
from collections import Counter

def analyze_tweet(tweet):
    result = {}
    result['MENTIONS'] = tweet.count('USER_MENTION')
    result['URLS'] = tweet.count('URL')
    result['POS_EMOS'] = tweet.count('EMO_POS')
    result['NEG_EMOS'] = tweet.count('EMO_NEG')
    tweet = tweet.replace('USER_MENTION', '').replace(
        'URL', '')
    words = tweet.split()
    result['WORDS'] = len(words)
    bigrams = get_bigrams(words)
    result['BIGRAMS'] = len(bigrams)
    return result, words, bigrams

def get_bigrams(tweet_words):
    bigrams = []
    num_words = len(tweet_words)
    for i in range(num_words - 1):
        bigrams.append((tweet_words[i], tweet_words[i + 1]))
    return bigrams

def get_bigram_freqdist(bigrams):
    freq_dict = {}
    for bigram in bigrams:
        if freq_dict.get(bigram):
            freq_dict[bigram] += 1
        else:
            freq_dict[bigram] = 1
    counter = Counter(freq_dict)
    return counter

In [12]:
# CALCULATE FREQUENCY DISTRIBUTIONS (for each features dataset)
num_tweets, num_pos_tweets, num_neg_tweets = 0, 0, 0
num_mentions, max_mentions = 0, 0
num_emojis, num_pos_emojis, num_neg_emojis, max_emojis = 0, 0, 0, 0
num_urls, max_urls = 0, 0
num_words, num_unique_words, min_words, max_words = 0, 0, 1e6, 0
num_bigrams, num_unique_bigrams = 0, 0
all_words = []
all_bigrams = []
datasets = ['word', 'swnt', 'pos']
for d in datasets:
    print(d)
    with open('dataset/' + d + '/' + d + '_features_train.csv', 'r') as csv:
        lines = csv.readlines()
        num_tweets = len(lines)
        for i, line in enumerate(lines):
            if i == 0:
                continue
            t_id, if_pos, tweet = line.strip().split(',')
            if_pos = int(if_pos)
            if if_pos:
                num_pos_tweets += 1
            else:
                num_neg_tweets += 1
            result, words, bigrams = analyze_tweet(tweet)
            num_mentions += result['MENTIONS']
            max_mentions = max(max_mentions, result['MENTIONS'])
            num_pos_emojis += result['POS_EMOS']
            num_neg_emojis += result['NEG_EMOS']
            max_emojis = max(
                max_emojis, result['POS_EMOS'] + result['NEG_EMOS'])
            num_urls += result['URLS']
            max_urls = max(max_urls, result['URLS'])
            num_words += result['WORDS']
            min_words = min(min_words, result['WORDS'])
            max_words = max(max_words, result['WORDS'])
            all_words.extend(words)
            num_bigrams += result['BIGRAMS']
            all_bigrams.extend(bigrams)
            sys.stdout.write('\r')
            sys.stdout.write('Processing %d/%d' % (i+1, num_tweets))
            sys.stdout.flush()
    num_emojis = num_pos_emojis + num_neg_emojis
    unique_words = list(set(all_words))
    with open('dataset/' + d + '/' + d + '_features_train_unique.txt', 'w') as uwf:
        uwf.write('\n'.join(unique_words))
    num_unique_words = len(unique_words)
    num_unique_bigrams = len(set(all_bigrams))
    print('\nCalculating frequency distribution')
    # Unigrams
    freq_dist = FreqDist(all_words)
    pkl_file_name = 'dataset/' + d + '/' + d + '_features_train_freqdist.pkl'
    with open(pkl_file_name, 'wb') as pkl_file:
        pickle.dump(freq_dist, pkl_file)
    print('Saved uni-frequency distribution to %s' % pkl_file_name)
    # Bigrams
    bigram_freq_dist = get_bigram_freqdist(all_bigrams)
    bi_pkl_file_name = 'dataset/' + d + '/' + d + '_features_train_freqdistbi.pkl'
    with open(bi_pkl_file_name, 'wb') as pkl_file:
        pickle.dump(bigram_freq_dist, pkl_file)
    print('Saved bi-frequency distribution to %s' % bi_pkl_file_name)
    print('\n[Analysis Statistics]')
    print('Tweets => Total: %d, Positive: %d, Negative: %d' % (num_tweets, num_pos_tweets, num_neg_tweets))
    print('User Mentions => Total: %d, Avg: %.4f, Max: %d' % (num_mentions, num_mentions / float(num_tweets), max_mentions))
    print('URLs => Total: %d, Avg: %.4f, Max: %d' % (num_urls, num_urls / float(num_tweets), max_urls))
    print('Emojis => Total: %d, Positive: %d, Negative: %d, Avg: %.4f, Max: %d' % (num_emojis, num_pos_emojis, num_neg_emojis, num_emojis / float(num_tweets), max_emojis))
    print('Words => Total: %d, Unique: %d, Avg: %.4f, Max: %d, Min: %d' % (num_words, num_unique_words, num_words / float(num_tweets), max_words, min_words))
    print('Bigrams => Total: %d, Unique: %d, Avg: %.4f' % (num_bigrams, num_unique_bigrams, num_bigrams / float(num_tweets)))
    print('\n')

word
Processing 99981/99990Processing 39001/99990
Calculating frequency distribution
Saved uni-frequency distribution to dataset/word/word_features_train_freqdist.pkl
Saved bi-frequency distribution to dataset/word/word_features_train_freqdistbi.pkl

[Analysis Statistics]
Tweets => Total: 99990, Positive: 56457, Negative: 43532
User Mentions => Total: 88955, Avg: 0.8896, Max: 12
URLs => Total: 4237, Avg: 0.0424, Max: 4
Emojis => Total: 1184, Positive: 997, Negative: 187, Avg: 0.0118, Max: 5
Words => Total: 1154343, Unique: 48477, Avg: 11.5446, Max: 93, Min: 0
Bigrams => Total: 1054840, Unique: 381798, Avg: 10.5495


swnt
Processing 99990/99990rocessing 10297/99990Processing 18695/99990Processing 47471/99990Processing 76651/99990
Calculating frequency distribution
Saved uni-frequency distribution to dataset/swnt/swnt_features_train_freqdist.pkl
Saved bi-frequency distribution to dataset/swnt/swnt_features_train_freqdistbi.pkl

[Analysis Statistics]
Tweets => Total: 99990, Positive: 1129

### Create features vectors

In [18]:
import numpy as np
import random
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer


def split_data(tweets, validation_split=0.1):
    index = int((1 - validation_split) * len(tweets))
    random.shuffle(tweets)
    return tweets[:index], tweets[index:]

def top_n_words(pkl_file_name, N, shift=0):
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    words = {p[0]: i + shift for i, p in enumerate(most_common)}
    return words

def top_n_bigrams(pkl_file_name, N, shift=0):
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    bigrams = {p[0]: i for i, p in enumerate(most_common)}
    return bigrams

def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


class FeatureVectorGenerator():
    
    def __init__(self, freq_dist_file, bi_freq_dist_file, train_processed_file, use_bigrams=True):
        self.FREQ_DIST_FILE = freq_dist_file
        self.BI_FREQ_DIST_FILE = bi_freq_dist_file
        self.TRAIN_PROCESSED_FILE = train_processed_file
        self.USE_BIGRAMS = use_bigrams
        self.UNIGRAM_SIZE = 15000
        self.VOCAB_SIZE = self.UNIGRAM_SIZE
        if use_bigrams:
            self.BIGRAM_SIZE = 10000
            self.VOCAB_SIZE = self.UNIGRAM_SIZE + self.BIGRAM_SIZE
    
    def get_feature_vector(self, tweet):
        # get the unigrams and bigrams contained in the vocabolary
        uni_feature_vector = []
        bi_feature_vector = []
        words = tweet.split()
        for i in range(len(words) - 1):
            word = words[i]
            next_word = words[i + 1]
            if self.unigrams.get(word):
                uni_feature_vector.append(word)
            if self.USE_BIGRAMS:
                if self.bigrams.get((word, next_word)):
                    bi_feature_vector.append((word, next_word))
        if len(words) >= 1:
            if self.unigrams.get(words[-1]):
                uni_feature_vector.append(words[-1])
        return uni_feature_vector, bi_feature_vector
    
    def extract_features(self, tweets, test_file=True, feat_type='presence'):
        # feat_type can be 'presence' for bag of words or frequency for tf-idf
        features = lil_matrix((len(tweets), self.VOCAB_SIZE))
        labels = np.zeros(len(tweets))
        for j, tweet in enumerate(tweets):
            sys.stdout.write('\r')
            sys.stdout.write('Processing %d/%d' % (j+1, len(tweets)))
            sys.stdout.flush()
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = self.unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if self.USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = self.bigrams.get(bigram)
                    if idx:
                        features[j, self.UNIGRAM_SIZE + idx] += 1
        if feat_type == 'frequency':
            if not test_file:
                self.tfidf = apply_tf_idf(features)
            features = self.tfidf.transform(features)
        return features, labels
    
    def extract_features_single(self, tweet, feat_type='presence'):
        features = lil_matrix((1, self.VOCAB_SIZE))
        tweet_words = tweet[0]
        tweet_bigrams = tweet[1]
        if feat_type == 'presence':
            tweet_words = set(tweet_words)
            tweet_bigrams = set(tweet_bigrams)
        for word in tweet_words:
            idx = self.unigrams.get(word)
            if idx:
                features[0, idx] += 1
        if self.USE_BIGRAMS:
            for bigram in tweet_bigrams:
                idx = self.bigrams.get(bigram)
                if idx:
                    features[0, self.UNIGRAM_SIZE + idx] += 1
        if feat_type == 'frequency':
            features = self.tfidf.transform(features)
        return features

    def process_tweets(self, csv_file, test_file=True):
        tweets = []
        print('Generating feature vectors')
        with open(csv_file, 'r') as csv:
            lines = csv.readlines()
            total = len(lines)
            for i, line in enumerate(lines):
                if i == 0:
                    # ignore header
                    continue
                if test_file:
                    tweet_id, tweet = line.split(',')
                else:
                    tweet_id, sentiment, tweet = line.split(',')
                feature_vector = self.get_feature_vector(tweet)
                if test_file:
                    tweets.append((tweet_id, feature_vector))
                else:
                    tweets.append((tweet_id, int(sentiment), feature_vector))
                sys.stdout.write('\r')
                sys.stdout.write('Processing %d/%d' % (i+1, total))
                sys.stdout.flush()
        print('\n')
        return tweets
    
    def fit(self):
        self.unigrams = top_n_words(self.FREQ_DIST_FILE, self.UNIGRAM_SIZE)
        if self.USE_BIGRAMS:
            self.bigrams = top_n_bigrams(self.BI_FREQ_DIST_FILE, self.BIGRAM_SIZE)

### Word model

In [19]:
FREQ_DIST_FILE = 'dataset/word/word_features_train_freqdist.pkl'
BI_FREQ_DIST_FILE = 'dataset/word/word_features_train_freqdistbi.pkl'
TRAIN_PROCESSED_FILE = 'dataset/word/word_features_train.csv'
FEAT_TYPE = 'frequency'
#TEST_PROCESSED_FILE = 'dataset/test-processed.csv'

word_feature_generator = FeatureVectorGenerator(FREQ_DIST_FILE, BI_FREQ_DIST_FILE, TRAIN_PROCESSED_FILE)
word_feature_generator.fit()
tweets = word_feature_generator.process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
train_tweets, val_tweets = split_data(tweets)

print('Extracting features & training batches')
word_training_set_X, word_training_set_y = word_feature_generator.extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE)
word_validation_set_X, word_validation_set_y = word_feature_generator.extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE)

Generating feature vectors
Processing 99990/99990Processing 21990/99990Processing 22790/99990Processing 23590/99990Processing 24390/99990Processing 34989/99990Processing 36589/99990Processing 61179/99990Processing 71376/99990

Extracting features & training batches
Processing 9999/999990rocessing 3982/89990Processing 5982/89990Processing 7982/89990Processing 9382/89990Processing 32179/89990Processing 37378/89990Processing 45377/89990Processing 60176/89990Processing 62776/89990Processing 83175/89990Processing 7980/9999

In [80]:
print(train_tweets[0])
print(word_validation_set_X[0])

('78779', 1, (['POS-2', 'it', 'was', 'a', 'super', 'sweet', 'movie', 'sounds', 'POS-1', 'there', 'NEG-2', 'madly', 'in', 'POS-6', 'that', 'POS-2'], [('POS-2', 'it'), ('it', 'was'), ('was', 'a'), ('a', 'super'), ('sounds', 'POS-1'), ('POS-1', 'there'), ('there', 'NEG-2'), ('in', 'POS-6'), ('POS-6', 'that'), ('that', 'POS-2')]))
  (0, 15247)	0.592939059870466
  (0, 15014)	0.4326711641295425
  (0, 252)	0.5189303024374169
  (0, 67)	0.3864506252084344
  (0, 2)	0.206364218068287


### SWNt model

In [78]:
FREQ_DIST_FILE = 'dataset/swnt/swnt_features_train_freqdist.pkl'
BI_FREQ_DIST_FILE = 'dataset/swnt/swnt_features_train_freqdistbi.pkl'
TRAIN_PROCESSED_FILE = 'dataset/swnt/swnt_features_train.csv'
FEAT_TYPE = 'frequency'
#TEST_PROCESSED_FILE = 'dataset/test-processed.csv'

swnt_feature_generator = FeatureVectorGenerator(FREQ_DIST_FILE, BI_FREQ_DIST_FILE, TRAIN_PROCESSED_FILE)
swnt_feature_generator.fit()
tweets = swnt_feature_generator.process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
train_tweets, val_tweets = split_data(tweets)

print('Extracting features & training batches')
swnt_training_set_X, swnt_training_set_y = swnt_feature_generator.extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE)
swnt_validation_set_X, swnt_validation_set_y = swnt_feature_generator.extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE)

Generating feature vectors
Processing 99990/99990Processing 21211/99990Processing 23811/99990Processing 44608/99990Processing 64596/99990Processing 75395/99990Processing 81394/99990Processing 85193/99990

Extracting features & training batches
Processing 9999/999990rocessing 5599/89990Processing 7797/89990Processing 34766/89990Processing 36565/89990Processing 50759/89990Processing 63552/89990Processing 89332/89990Processing 5534/9999

In [82]:
print(train_tweets[0])
print(swnt_training_set_X[0])

('78779', 1, (['POS-2', 'it', 'was', 'a', 'super', 'sweet', 'movie', 'sounds', 'POS-1', 'there', 'NEG-2', 'madly', 'in', 'POS-6', 'that', 'POS-2'], [('POS-2', 'it'), ('it', 'was'), ('was', 'a'), ('a', 'super'), ('sounds', 'POS-1'), ('POS-1', 'there'), ('there', 'NEG-2'), ('in', 'POS-6'), ('POS-6', 'that'), ('that', 'POS-2')]))
  (0, 19856)	0.3147555916391844
  (0, 18103)	0.27663036188226203
  (0, 17671)	0.2746992632891251
  (0, 16485)	0.25526056973342376
  (0, 15567)	0.22954685806630287
  (0, 15295)	0.2121164155183353
  (0, 15176)	0.20163291599171174
  (0, 15128)	0.19326239710717036
  (0, 15101)	0.21102998661198608
  (0, 15015)	0.17602099386521589
  (0, 9748)	0.3648116420498138
  (0, 458)	0.23277924768086494
  (0, 298)	0.2145579309778834
  (0, 250)	0.20803095995953877
  (0, 180)	0.19634791928677564
  (0, 54)	0.15266627690155346
  (0, 24)	0.1257952497531284
  (0, 14)	0.10915564731868259
  (0, 13)	0.08798804917420781
  (0, 12)	0.08758334429472282
  (0, 11)	0.10829652807431024
  (0, 7)	0.

### POS+POS_word model

In [42]:
FREQ_DIST_FILE = 'dataset/pos/pos_features_train_freqdist.pkl'
BI_FREQ_DIST_FILE = 'dataset/pos/pos_features_train_freqdistbi.pkl'
TRAIN_PROCESSED_FILE = 'dataset/pos/pos_features_train.csv'
FEAT_TYPE = 'frequency'
#TEST_PROCESSED_FILE = 'dataset/test-processed.csv'

pos_feature_generator = FeatureVectorGenerator(FREQ_DIST_FILE, BI_FREQ_DIST_FILE, TRAIN_PROCESSED_FILE)
pos_feature_generator.fit()
tweets = pos_feature_generator.process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
train_tweets, val_tweets = split_data(tweets)

print('Extracting features & training batches')
pos_training_set_X, pos_training_set_y = pos_feature_generator.extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE)
pos_validation_set_X, pos_validation_set_y = pos_feature_generator.extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE)

Generating feature vectors
Processing 99990/99990Processing 16919/99990Processing 18718/99990Processing 34114/99990Processing 38313/99990Processing 50912/99990Processing 58511/99990Processing 62564/99990Processing 83709/99990Processing 84509/99990Processing 95708/99990Processing 99108/99990

Extracting features & training batches
Processing 9999/999990Processing 27486/89990Processing 35276/89990Processing 42868/89990Processing 54457/89990

In [75]:
print(train_tweets[3])
print(pos_training_set_X[3])

('45109', 1, (['NNP', 'VBZ', 'JJ', 'NNS', 'VBD', 'WRB', 'JJ', 'PRP', 'VBZ_lmfao', 'WRB_how', 'JJ_awesome', 'NN_r', 'PRP_we'], [('NNP', 'VBZ'), ('VBZ', 'JJ'), ('JJ', 'NNS'), ('NNS', 'VBD'), ('VBD', 'WRB'), ('WRB', 'JJ'), ('JJ', 'NN'), ('NN', 'PRP')]))
  (0, 16439)	0.3512236167870001
  (0, 15243)	0.2640284394346589
  (0, 15206)	0.2575266294810932
  (0, 15062)	0.19982924699753002
  (0, 15048)	0.18999028698687326
  (0, 15025)	0.1689249765765872
  (0, 15023)	0.16941112421120189
  (0, 15001)	0.09968915875379583
  (0, 7324)	0.4544510532691738
  (0, 1180)	0.3434394046883337
  (0, 524)	0.28676467036022707
  (0, 185)	0.22622405922154876
  (0, 149)	0.21310038411184548
  (0, 76)	0.1741452751888806
  (0, 21)	0.11476039645517354
  (0, 17)	0.11098404403383368
  (0, 14)	0.09839175847759304
  (0, 9)	0.07721625319403355
  (0, 5)	0.0828498871447997
  (0, 1)	0.1256519500362839


### preprocess on test tweets

In [20]:
sentence = 'Hi, this is an awesome sample tweet to try how testing wooooorks! :) car'
prep = preprocess_tweet(sentence)
sentence_prep = word_feature_generator.get_feature_vector(prep)
print(sentence_prep)
print(word_feature_generator.extract_features_single(sentence_prep, feat_type='frequency'))

(['hi', 'this', 'is', 'an', 'awesome', 'sample', 'tweet', 'to', 'try', 'how', 'testing', 'EMO_POS', 'car'], [('this', 'is'), ('is', 'an'), ('an', 'awesome'), ('tweet', 'to'), ('to', 'try')])
  (0, 24259)	0.3376941068180862
  (0, 16125)	0.29697791202285495
  (0, 15769)	0.2806029373441025
  (0, 15723)	0.27804595957867934
  (0, 15095)	0.22989261228624
  (0, 5461)	0.3270321841162721
  (0, 2179)	0.30629199609262525
  (0, 411)	0.24458434805598597
  (0, 204)	0.21647210763558936
  (0, 201)	0.21509077466989512
  (0, 195)	0.21790692466395373
  (0, 157)	0.20810836993518436
  (0, 131)	0.19716673664030696
  (0, 92)	0.17971289177657207
  (0, 54)	0.1687908874624308
  (0, 34)	0.15150167112019972
  (0, 9)	0.11670681571298516
  (0, 2)	0.09016934357250361
